# Info agregada por Cele

In [1]:

tweet_totals = dataset['longitud'].count()
tweet_true_count = tweet_len_true.count()
tweet_false_count = tweet_len_false.count()

labels = 'True Tweets', 'False Tweets'
sizes = [tweet_true_count, tweet_false_count]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=False, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

NameError: name 'dataset' is not defined

## Análisis de los hashtags

In [ ]:
# extrae los hashtags de un texto y devuelve una lista 
def hashtags(text):
    result = []
    for token in text.split():
        if token[0]=='#':
            result.append(token.replace('#', '').lower())
    return result    

text = 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'
hashtags(text)

In [ ]:
dataset['hashtags'] = dataset.text.apply(lambda x : hashtags(x))
dataset.head(5)

In [ ]:
df_hashtags = dataset.explode('hashtags')
df_hashtags.head(20)

In [ ]:
# hay que ver si vale la pena porque hay muy pocos hashtags en comparacion al tamaño del dataset
df_hashtags.info()

In [ ]:
top_hashtags_series = df_hashtags.hashtags.value_counts().nlargest(50)
top_hashtags = pd.DataFrame(top_hashtags_series).reset_index()
top_hashtags.columns = ['hashtags','Cantidad']
top_hashtags.head()


In [ ]:
top_hashtags.Cantidad.sum()

In [ ]:

fig, ax = plt.subplots()


plt.xlabel("Hashtags",fontsize=14)
plt.ylabel('Cantidad',fontsize=14)
plt.title('Análisis de los hashtags más frecuentes',fontsize=16,fontweight='bold', pad = 15)

top_hashtags.plot.bar(x='hashtags',y='Cantidad', ax = ax)

plt.show()

## Análisis por ubicacion

## Análisis por keywords

In [ ]:
dataset.keyword.value_counts()

In [ ]:
dataset.keyword.isnull().value_counts()

In [ ]:
top_keywords = dataset.keyword.value_counts().reset_index()
top_keywords.columns = ['Keywords', 'Cantidad']

top_keywords.head(5)

In [ ]:
top_keywords.describe()
# estan distribuidas bastante uniformemente

In [ ]:
fig, ax = plt.subplots()

top_keywords.nlargest(50,'Cantidad').plot(kind='bar', x = 'Keywords', ax = ax)

plt.xlabel("Keywords",fontsize=14)
plt.ylabel('Cantidad',fontsize=14)
plt.title('Análisis de las keywords más frecuentes',fontsize=16,fontweight='bold', pad = 15)


#agrego texto para probar cambios 

# TEXTO 2 PARA VER HISTORIAL 

# ahora esto lo agrego con mi usuario 


# Info agregada por ...

In [ ]:
# Pongan toda la info en celdas como estas